<a id='Centre-Right-Scraping-and-cleaning'></a>

In [163]:
import requests
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
#from langdetect import detect

In [164]:
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000

In [165]:
np.set_printoptions(threshold=np.inf)
pd.options.display.max_colwidth = 1000

- List of people I considered for this category

In [27]:
lista = ['William F. Buckley Jr.',
         'George Will', 
         "Bill O'Reilly", 
         'Barry Goldwater',
         'Andrew Sullivan', 
         'Robert P George', 
         'Roger Stone',
         'Paul Manafort', 
         'Milton Friedman',
         'Thomas Sowell',
         'Charles Murray', 
         'Kevin D Williamson',
         'Walter E Williams', 
         'Robert Lucas Jr',
         'Andrew Breitbart', 
         'Bill Kristol',
         'David Friedman', 
         'William Happer', 
         'Ben Stein', 
         'Glenn Beck',
         'Mona Charen',
         'David Frum', 
         'David Horowitz', 
         'Jeane Kirkpatrick',
         'Charles Krauthammer', 
         'Irving Kristol', 
         'W. Cleon Skousen',
         'Rush Limbaugh',
         'Tucker Carlson',
         'Andrew Napolitano',
         'Roger Kimball',
         'Michelle Mallon',
         'David Clarke']

In [31]:
temp_lista = []
new_lista = []

In [32]:
for name in lista:
    temp_lista.append(name.replace('.',''))

In [33]:
for each in temp_lista:
    new_lista.append(each.replace(' ','+'))


In [34]:
new_lista

['William+F+Buckley+Jr',
 'George+Will',
 "Bill+O'Reilly",
 'Barry+Goldwater',
 'Andrew+Sullivan',
 'Robert+P+George',
 'Roger+Stone',
 'Paul+Manafort',
 'Milton+Friedman',
 'Thomas+Sowell',
 'Charles+Murray',
 'Kevin+D+Williamson',
 'Walter+E+Williams',
 'Robert+Lucas+Jr',
 'Andrew+Breitbart',
 'Bill+Kristol',
 'David+Friedman',
 'William+Happer',
 'Ben+Stein',
 'Glenn+Beck',
 'Mona+Charen',
 'David+Frum',
 'David+Horowitz',
 'Jeane+Kirkpatrick',
 'Charles+Krauthammer',
 'Irving+Kristol',
 'W+Cleon+Skousen',
 'Rush+Limbaugh',
 'Tucker+Carlson',
 'Andrew+Napolitano',
 'Roger+Kimball',
 'Michelle+Mallon',
 'David+Clarke']

In [35]:
#Scraping all the existing quotes for the person I need from Goodreads
quotes = []
author = []

for name in lista:
    second_temp_quote = []
    second_temp_author = []
    r = requests.get("https://goodquotesapi.herokuapp.com/author/{}".format(name))
    data = r.json()
    if len(data) < 1:
        pass
    else:
        for page in tqdm_notebook(range(1,data['total_pages']+1)):
            new_r = requests.get("https://goodquotesapi.herokuapp.com/author/{}?page={}".format(name,page))
            new_data = new_r.json()
            first_temp_quote = []
            first_temp_author = []

            for i in range(20):
                temp_quote = []
                temp_author = []

                try: 
                    temp_quote.append(new_data['quotes'][i]['quote'])
                    temp_author.append(new_data['quotes'][0]['author'])

                except:
                    temp_quote.append(np.nan)
                    temp_author.append(np.nan)
                first_temp_author.append(temp_author)
                first_temp_quote.append(temp_quote)
            second_temp_quote.append(first_temp_quote)
            second_temp_author.append(first_temp_author)
    quotes.append(second_temp_quote)
    author.append(second_temp_author)

In [36]:
#Creating the goodreads Dataframe
goodreads = pd.DataFrame({
    'Author': [item for l in author for sublist in l for j in l for item in sublist], 
    'Quotes': [item for l in quotes for sublist in l for j in l for item in sublist],
    'source': 'goodreads'
})

In [37]:
goodreads.head()

,Author,Quotes,source
0,[William F. Buckley Jr.],[I won't insult your intelligence by suggesting that you really believe what you just said.],goodreads
1,[William F. Buckley Jr.],[I would rather be governed by the first 2000 people in the Manhattan phone book than the entire faculty of Harvard.],goodreads
2,[William F. Buckley Jr.],[Life can't be all bad when for ten dollars you can buy all the Beethoven sonatas and listen to them for ten years.],goodreads
3,[William F. Buckley Jr.],"[Liberals, it has been said, are generous with other peoples' money, except when it comes to questions of national survival when they prefer to be generous with other peoples' freedom and security.]",goodreads
4,[William F. Buckley Jr.],[I would like to take you seriously but to do so would affront your intelligence.],goodreads


In [38]:
#Leaving every cell in a string
goodreads.Author = goodreads.Author.apply(lambda x : x[0])
goodreads.Quotes = goodreads.Quotes.apply(lambda x : x[0])

In [39]:
#Deleting all the rows where the Author is not in the list
goodreads = goodreads[goodreads['Author'].isin(lista)]

- Wikiquote

In [41]:
#Loading the previous dataframe scraped from Wikiquotes
wikiquotes = pd.read_csv('../WikiQuote/centre_right.csv')

In [43]:
import re
#regex = re.compile(".*?\((.*?)\)")

In [44]:
wikiquotes.Author = wikiquotes.Author.apply(lambda x : re.sub(r'\(.*?\)', '', x))

In [47]:
wikiquotes = wikiquotes.drop('Political Orientation', axis=1)

In [48]:
wikiquotes['source'] = 'wikiquotes'

- BrainQuote

In [49]:
#Creating a list for the Brainquote format
brain_lista = []
for n in temp_lista:
    brain_lista.append(n.lower().replace(' ','_'))
    

In [50]:
brain_lista

['william_f_buckley_jr',
 'george_will',
 "bill_o'reilly",
 'barry_goldwater',
 'andrew_sullivan',
 'robert_p_george',
 'roger_stone',
 'paul_manafort',
 'milton_friedman',
 'thomas_sowell',
 'charles_murray',
 'kevin_d_williamson',
 'walter_e_williams',
 'robert_lucas_jr',
 'andrew_breitbart',
 'bill_kristol',
 'david_friedman',
 'william_happer',
 'ben_stein',
 'glenn_beck',
 'mona_charen',
 'david_frum',
 'david_horowitz',
 'jeane_kirkpatrick',
 'charles_krauthammer',
 'irving_kristol',
 'w_cleon_skousen',
 'rush_limbaugh',
 'tucker_carlson',
 'andrew_napolitano',
 'roger_kimball',
 'michelle_mallon',
 'david_clarke']

In [51]:
#Scraping the quotes from Brainquote website
q = []
a = []

for name in brain_lista:
    temp_quote = []
    temp_author = []
    r = requests.get("https://www.brainyquote.com/authors/{}".format(name))
    soup = BeautifulSoup(r.text,'html.parser')
    for each in soup.find_all('a', attrs={'title':"view quote"}):
        temp_quote.append(each.text)
        #if len(temp_quote) > 0:
        temp_author.append(name)
    q.append(temp_quote)
    a.append(temp_author)
    


In [52]:
#Creating the Brainquote dataframe
brain = pd.DataFrame({'Author': [item for l in a for item in l],
                      'Quotes': [item for l in q for item in l],
                      'source': 'brainquote'})

In [53]:
#Removing '_' from the name and capitalize Name and Surname
brain.Author = brain.Author.apply(lambda x : x.replace('_',' ').title())

In [59]:
#Putting everything together
Centre_Right_quotes = pd.concat([goodreads, brain, wikiquotes])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [60]:
Centre_Right_quotes = Centre_Right_quotes.drop_duplicates(subset='Quotes')

In [62]:
Centre_Right_quotes = Centre_Right_quotes.dropna()

- Detecting the language

In [76]:
Centre_Right_quotes['language'] = Centre_Right_quotes.Quotes.apply(lambda x : detect(x))

In [77]:
Centre_Right_quotes.language.value_counts()

en    3027
es      97
af       9
fr       7
de       5
it       4
ca       4
so       4
cy       4
sv       3
id       3
cs       3
da       2
no       2
sk       2
nl       1
ro       1
sw       1
tr       1
ar       1
lv       1
lt       1
tl       1
Name: language, dtype: int64

In [78]:
Centre_Right_quotes = Centre_Right_quotes[Centre_Right_quotes.language == 'en']

- Changing the name of few authors

In [80]:
Centre_Right_quotes.loc[Centre_Right_quotes['Author'].str.contains('Buckley'), 'Author'] = 'William F. Buckley Jr.'
Centre_Right_quotes.loc[Centre_Right_quotes['Author'].str.contains('Robert Lucas'), 'Author'] = 'Robert Lucas Jr.'
Centre_Right_quotes.loc[Centre_Right_quotes['Author'].str.contains('Reilly'), 'Author'] = "Bill O'Really"


- Looking for quote in the quotes

In [81]:
dup = {}
#count = 0
for name in Centre_Right_quotes.Author.unique():
    conta = []
    for text in Centre_Right_quotes[Centre_Right_quotes.Author==name].Quotes :
        count = 0
        for i in range(len(Centre_Right_quotes[Centre_Right_quotes.Author==name])):
            if text in Centre_Right_quotes[Centre_Right_quotes.Author==name].iloc[i,1]:
                count += 1


        conta.append(count)
    dup[name] = conta

In [82]:
Centre_Right_quotes['Counter'] = 0
for name in Centre_Right_quotes.Author.unique():
    print(name, ' done')
    Centre_Right_quotes.loc[Centre_Right_quotes.Author==name, 'Counter'] = dup[name]


William F. Buckley Jr.  done
Bill O'Really  done
Roger Stone  done
Milton Friedman  done
Thomas Sowell  done
Charles Murray  done
Glenn Beck  done
David Frum  done
David Horowitz  done
Charles Krauthammer  done
Irving Kristol  done
W. Cleon Skousen  done
Rush Limbaugh  done
Roger Kimball  done
George Will  done
Barry Goldwater  done
Andrew Sullivan  done
Paul Manafort  done
Robert Lucas Jr.  done
Andrew Breitbart  done
Bill Kristol  done
David Friedman  done
Ben Stein  done
Jeane Kirkpatrick  done
Tucker Carlson  done
Robert P. George  done
Charles Murray   done
Kevin D. Williamson  done
Walter E. Williams  done
William Happer  done
Mona Charen  done


In [84]:
Centre_Right_quotes.Counter.value_counts()

1      2968
2        50
3         5
4         2
103       1
6         1
Name: Counter, dtype: int64

In [86]:
Centre_Right_quotes=Centre_Right_quotes[Centre_Right_quotes.Counter==1]

In [88]:
Centre_Right_quotes.drop(['source', 'language', 'Counter'], axis=1, inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [92]:
#Adding the political orientation
Centre_Right_quotes['Political Orientation'] = 'Conservative'

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [96]:
#Saving
Centre_Right_quotes.to_csv('centre_right_quotes.csv', index=False)

In [98]:
#Selecting 4 Authors for a total of bit more than 100 quotes for the final test df
list_test = ['David Friedman', 'Tucker Carlson', 'Andrew Breitbart', 'Robert Lucas Jr.']

In [99]:
#Creating the Dataframe for the test
CR_test = Centre_Right_quotes[Centre_Right_quotes['Author'].isin(list_test)]


In [100]:
CR_test.Author.value_counts()


David Friedman      37
Andrew Breitbart    30
Tucker Carlson      25
Robert Lucas Jr.    16
Name: Author, dtype: int64

In [101]:
#removing the 4 authors from the main database
Centre_Right_quotes = Centre_Right_quotes[~Centre_Right_quotes['Author'].isin(list_test)]

In [105]:
#csv file for the train
Centre_Right_quotes.to_csv('train_centre_right.csv', index=False)

In [104]:
#csv file for the test
CR_test.to_csv('test_centre_right.csv', index=False)

**SECOND PART**

In [4]:
#Loading files

In [87]:
CR_train = pd.read_csv('train_centre_right.csv')

In [88]:
CR_test = pd.read_csv('test_centre_right.csv')

- Looking for quotes about the author and not by the author

In [90]:
def find_autoref(df):
    autoref = {}
    for name in df.Author.unique():
        conta = []

        for text in df[df.Author==name].Quotes :
            count = 0

            if name in text or name.split(' ')[-1] in text:
                    count += 1

            conta.append(count)
        autoref[name] = conta 
    return autoref

In [91]:
autoref = find_autoref(CR_train)

In [92]:
CR_train['auto'] = 0
for name in CR_train.Author.unique():
    print(name, ' done')
    CR_train.loc[CR_train.Author==name, 'auto'] = autoref[name]

William F. Buckley Jr.  done
Bill O'Really  done
Roger Stone  done
Milton Friedman  done
Thomas Sowell  done
Charles Murray  done
Glenn Beck  done
David Frum  done
David Horowitz  done
Charles Krauthammer  done
Irving Kristol  done
W. Cleon Skousen  done
Rush Limbaugh  done
Roger Kimball  done
George Will  done
Barry Goldwater  done
Andrew Sullivan  done
Paul Manafort  done
Bill Kristol  done
Ben Stein  done
Jeane Kirkpatrick  done
Robert P. George  done
Charles Murray   done
Kevin D. Williamson  done
Walter E. Williams  done
William Happer  done
Mona Charen  done


In [93]:
CR_train.auto.sum()

125

In [94]:
CR_train=CR_train[CR_train.auto==0]

In [95]:
from textacy.preprocess import preprocess_text
import re
#import nltk

In [96]:
nans = lambda df: df[df.isnull().any(axis=1)]
nans(CR_train)

,Author,Quotes,Political Orientation,auto


In [105]:
CR_train['texicity'] = CR_train.Quotes.apply(lambda x : preprocess_text(x, 
                                                                      lowercase=True, 
                                                                      no_punct=True, 
                                                                      no_accents=True, 
                                                                      no_contractions=True
                                                                       ))

In [106]:
CR_train.head()

,Author,Quotes,Political Orientation,auto,texicity
0,William F. Buckley Jr.,I won't insult your intelligence by suggesting that you really believe what you just said.,Conservative,0,i will not insult your intelligence by suggesting that you really believe what you just said
1,William F. Buckley Jr.,I would rather be governed by the first 2000 people in the Manhattan phone book than the entire faculty of Harvard.,Conservative,0,i would rather be governed by the first 2000 people in the manhattan phone book than the entire faculty of harvard
2,William F. Buckley Jr.,Life can't be all bad when for ten dollars you can buy all the Beethoven sonatas and listen to them for ten years.,Conservative,0,life can not be all bad when for ten dollars you can buy all the beethoven sonatas and listen to them for ten years
4,William F. Buckley Jr.,I would like to take you seriously but to do so would affront your intelligence.,Conservative,0,i would like to take you seriously but to do so would affront your intelligence
5,William F. Buckley Jr.,Back in the thirties we were told we must collectivize the nation because the people were so poor. Now we are told we must collectivize the nation because the people are so rich.,Conservative,0,back in the thirties we were told we must collectivize the nation because the people were so poor now we are told we must collectivize the nation because the people are so rich


In [107]:
from nltk.corpus import stopwords


In [108]:
stops = stopwords.words('english')

In [110]:
def tok_stop(df):
    df['texicity'] = df['texicity'].apply(lambda x: x.split())
    df['texicity'] = df['texicity'].apply(lambda x : [item for item in x if item not in stops])
    df['texicity'] = df['texicity'].apply(lambda x: ' '.join(x))
    df['texicity'] = df['texicity'].apply(lambda x: re.sub(r'\d+', '',x)) 
    return df
    

In [111]:
CR_train = tok_stop(CR_train)

- Looking for quote in the quote with token

In [112]:
fake = {}
#count = 0
for name in CR_train.Author.unique():
    print(name)
    conta = []
    for j,text in enumerate(CR_train[CR_train.Author==name].texicity):
        print(j)
        count = 0
        for i in range(len(CR_train[CR_train.Author==name])):
            if text in CR_train[CR_train.Author==name].iloc[i,4]:
                count += 1


        conta.append(count)
    fake[name] = conta

William F. Buckley Jr.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
Bill O'Really
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
Andrew Sullivan
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
Paul Manafort
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
Bill Kristol
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
Ben Stein
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
Jeane Kirkpatrick
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
Robert P. George
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41

In [113]:
CR_train['rep'] = 0
for name in CR_train.Author.unique():
    print(name, ' done')
    CR_train.loc[CR_train.Author==name, 'rep'] = fake[name]


William F. Buckley Jr.  done
Bill O'Really  done
Roger Stone  done
Milton Friedman  done
Thomas Sowell  done
Charles Murray  done
Glenn Beck  done
David Frum  done
David Horowitz  done
Charles Krauthammer  done
Irving Kristol  done
W. Cleon Skousen  done
Rush Limbaugh  done
Roger Kimball  done
George Will  done
Barry Goldwater  done
Andrew Sullivan  done
Paul Manafort  done
Bill Kristol  done
Ben Stein  done
Jeane Kirkpatrick  done
Robert P. George  done
Kevin D. Williamson  done
Walter E. Williams  done
William Happer  done
Mona Charen  done


In [114]:
CR_train.rep.value_counts()

1     2630
2       70
25       1
5        1
3        1
16       1
Name: rep, dtype: int64

In [115]:
CR_train.to_csv('CR_train_clean.csv', index=False)

In [116]:
CR_test=CR_test.drop(CR_test.index[34:59])

In [117]:
autoref_test=find_autoref(CR_test)

In [118]:
CR_test['auto'] = 0
for name in CR_test.Author.unique():
    print(name, ' done')
    CR_test.loc[CR_test.Author==name, 'auto'] = autoref_test[name]

Robert Lucas Jr.  done
Andrew Breitbart  done
David Friedman  done
Tucker Carlson  done


In [119]:
CR_test.auto.sum()

0

In [120]:
CR_test['texicity'] = CR_test.Quotes.apply(lambda x : preprocess_text(x, 
                                                                      lowercase=True, 
                                                                      no_punct=True, 
                                                                      no_accents=True, 
                                                                      no_contractions=True))

In [121]:
CR_test=tok_stop(CR_test)

- Looking for quote in the quote in the test

In [123]:
fake = {}
#count = 0
for name in CR_test.Author.unique():
    print(name)
    conta = []
    for i,text in enumerate(CR_test[CR_test.Author==name].texicity) :
        print(i)
        count = 0
        for i in range(len(CR_test[CR_test.Author==name])):
            if text in CR_test[CR_test.Author==name].iloc[i,4]:
                count += 1


        conta.append(count)
    fake[name] = conta

Robert Lucas Jr.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Andrew Breitbart
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
David Friedman
0
1
2
3
4
5
6
7
8
9
10
11
Tucker Carlson
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [124]:
CR_test['rep'] = 0
for name in CR_test.Author.unique():
    print(name, ' done')
    CR_test.loc[CR_test.Author==name, 'rep'] = fake[name]


Robert Lucas Jr.  done
Andrew Breitbart  done
David Friedman  done
Tucker Carlson  done


In [125]:
CR_test.rep.value_counts()

1    83
Name: rep, dtype: int64

In [126]:
CR_test.to_csv('CR_test_clean.csv', index=False)

**Third**

In [127]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

In [ ]:
from sklearn.feature_extraction import text 

stop_words = text.ENGLISH_STOP_WORDS

In [ ]:
stoppe = list(stop_words)

In [ ]:
def tok_stop(df):
    df['texicity'] = df['texicity'].apply(lambda x: x.split())
    df['texicity'] = df['texicity'].apply(lambda x : [item for item in x if item not in stoppe])
    df['texicity'] = df['texicity'].apply(lambda x: ' '.join(x))
    return df

In [ ]:
CR_train=tok_stop(CR_train)


In [135]:
def bag_word(df, orientation, mm, mx, col):
    for i in range(mm,mx):
        vect = CountVectorizer(stop_words='english', ngram_range=(i,i))
        words = {}
        lefty = df[df['Political Orientation']==orientation]
        Left_Voc = ''.join(lefty[col])
        lefty_n_grams = vect.build_analyzer()(Left_Voc)
        words['Left'] = Counter(lefty_n_grams).most_common()
        print('Number of gram: ', i)
        print(Counter(lefty_n_grams).most_common())
        print()
        print()
        return Counter(lefty_n_grams).most_common()

- Looking for quote in the quote with n_gram

In [ ]:
list_ind=[]
for gram in list_gram:
    if gram[1]>1:
        for text in CR_train['texicity']:
            
            try:
                if gram[0] in text:
                    CR_train.loc[CR_train['texicity']]
                    (one,two)=(CR_train.loc[CR_train['texicity'].str.contains(gram[0]), 'texicity'].index)
                if len(CR_train.loc[one,'texicity'])>len(CR_train.loc[two,'texicity']):
                    list_ind.append(two)
                else:
                    list_ind.append(one)

            except:
                pass

In [1]:
l=list(set(list_ind))


In [157]:
CR_train=CR_train.drop(l)


In [ ]:
get_stop_words(self)

In [183]:
l_2=[543, 546, 547, 1440, 1461, 1437, 2687, 1470, 1514, 1517, 1522, 1525, 1539, 1577, 1547,
    1909, 2288, 1946, 484, 2241, 382, 490, 1804, 2280, 2278, 531, 1814, 1994, 2650, 552, 426, 
     526, 975, 984, 3, 15, 1664, 2417, 82, 292, 348, 540, 2252, 2265, 2279, 500, 369, 2268, 455 ]

In [184]:
CR_train=CR_train.drop(l_2)


In [2]:
l3=list(set(list_ind))


In [188]:
CR_train=CR_train.drop(l3)


In [202]:
CR_train.to_csv('CR_train_clean.csv')